In [36]:
#This is my project for movie sentiment analysis. 
#i will be using NAIVE-BAYES THEOREM(A simplified version of the Bayes Theorem, known as the Naive Bayes Classification) to make the prediction
#the steps will be 1)text pre procesing , 1)vectorizing , 3)training the mode


In [37]:
import numpy as np 
import pandas as pd

In [38]:
df=pd.read_csv('IMDB Dataset.csv')


In [39]:
#for text processingn i will follow these steps
#remove html tag
#remove special char
#covert all to lower
#remove stop word
#stemming


In [40]:


df['sentiment'].replace({'positive':1,'negative':0},inplace=True)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [41]:
import re

In [42]:
def clean_html(text):
    clean=re.compile('<.*?>')
    return re.sub(clean,'',text)
    

In [43]:
def convert_low(text):
    return text.lower()

In [44]:
def remove_special(text):
    x=''
    for i in text:
        if i.isalnum():
            x=x+i
        else:
            x=x+' '
    return x

In [45]:
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords

In [46]:
def remove_stopper(text):
    x=[]
    for i in text.split():
        if i not in stopwords.words('english'):
            x.append(i)
    y=x[:]
    x.clear()
    return y

In [47]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [48]:
y=[]
def stem_words(text):
    for i in text:
        y.append(ps.stem(i))
    z=y[:]
    y.clear()
    return z

In [49]:
def join_back(list_input):
    return " ".join(list_input)

In [50]:
df['review']=df['review'].apply(clean_html)
df['review']=df['review'].apply(convert_low)
df['review']=df['review'].apply(remove_special)
df['review']=df['review'].apply(remove_stopper)
df['review']=df['review'].apply(stem_words)
df['review']=df['review'].apply(join_back)

In [51]:
df

,review,sentiment
0,one review mention watch 1 oz episod hook righ...,1
1,wonder littl product film techniqu unassum old...,1
2,thought wonder way spend time hot summer weeke...,1
3,basic famili littl boy jake think zombi closet...,0
4,petter mattei love time money visual stun film...,1
...,...,...
49995,thought movi right good job creativ origin fir...,1
49996,bad plot bad dialogu bad act idiot direct anno...,0
49997,cathol taught parochi elementari school nun ta...,0
49998,go disagre previou comment side maltin one sec...,0


In [52]:
#X=df.iloc[:,0:1].values
#print(X)
#X.shape


In [53]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=100)
X=cv.fit_transform(df['review']).toarray()

In [54]:
Y=df.iloc[:,-1].values

In [55]:
X.shape

(50000, 100)

In [56]:
Y.shape


(50000,)

In [57]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.02)
                               

In [58]:
X_train.shape

(49000, 100)

In [59]:
Y_test.shape

(1000,)

In [60]:
from sklearn.naive_bayes import GaussianNB,BernoulliNB
clf1=GaussianNB()
clf2=BernoulliNB()

In [61]:
clf1.fit(X_train,Y_train)

GaussianNB()

In [62]:
clf2.fit(X_train,Y_train)

BernoulliNB()

In [63]:
Y_pred1=clf1.predict(X_test)

In [64]:
Y_pred2=clf2.predict(X_test)

In [65]:
from sklearn.metrics import accuracy_score

In [66]:
print("Gaussian",accuracy_score(Y_test,Y_pred1))
print("Bernoulli",accuracy_score(Y_test,Y_pred2))

Gaussian 0.708
Bernoulli 0.714


In [67]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier

# Vectorize the text data using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=300, stop_words='english')
X_tfidf = tfidf_vectorizer.fit_transform(df['review'])

# Split the data into training and testing sets
X_train_tfidf, X_test_tfidf, Y_train, Y_test = train_test_split(X_tfidf, df['sentiment'], test_size=0.2, random_state=2022,stratify=df.sentiment)

# Initialize k-NN classifier
knn_classifier = KNeighborsClassifier(n_neighbors=5)  

# Train the k-NN classifier
knn_classifier.fit(X_train_tfidf, Y_train)

# Predict sentiment on the test set
Y_result_knn = knn_classifier.predict(X_test_tfidf)

# Evaluate accuracy
accuracy_knn = accuracy_score(Y_test, Y_result_knn)
print("Accuracy (k-NN):", accuracy_knn)


Accuracy (k-NN): 0.699


In [68]:
from sklearn.pipeline import Pipeline

clf = Pipeline([
     ('vectorizer_tfidf',TfidfVectorizer()),    
     ('KNN', KNeighborsClassifier())         
])
X_train, X_test, y_train, y_test = train_test_split(
    df.review, 
    df.sentiment, 
    test_size=0.2, # 20% samples will go to test dataset
    random_state=2022,
    stratify=df.sentiment
)



In [69]:
clf.fit(X_train, y_train)

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('KNN', KNeighborsClassifier())])

In [70]:
y_pred = clf.predict(X_test)


accuracy_knn = accuracy_score(y_test, y_pred)
print("Accuracy (k-NN):", accuracy_knn)

Accuracy (k-NN): 0.7798
